In [2]:
# load data and write out sentence and target
import pandas as pd

loaded_set = pd.read_excel("Dataset/"+"training.xlsx")
loaded_set['Sentence'][0]

'Weitergehende Sicherungsmaßnahmen können eine Videoüberwachung und eine Zugangskontrolle durch einen Türöffner sein, denn viele GAA befinden sich in Vorräumen der Geschäftsstellen der Banken, sodass sie auch außerhalb der Schalteröffnungszeiten zugänglich sind.'

In [3]:
import tensorflow_hub as hub
from transformers import AutoModel, AutoTokenizer
# german tokens for bert
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")



tokens_num=[]
for sen in loaded_set['Sentence']:
    tokenized = (tokenizer.tokenize(sen)) 
    tokens_num.append( ['[CLS]'] + tokenized + ['[SEP]']) 
    
# get max_seq length    
lens = [len(i) for i in tokens_num]
max_seq_length = max(lens)
max_seq_length = int(1.5*max_seq_length)
tokens_num[0]

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


['[CLS]',
 'Weiter',
 '##gehende',
 'Sicherungs',
 '##maßnahmen',
 'können',
 'eine',
 'Video',
 '##überwachung',
 'und',
 'eine',
 'Zugangs',
 '##kontrolle',
 'durch',
 'einen',
 'Tür',
 '##öff',
 '##ner',
 'sein',
 ',',
 'denn',
 'viele',
 'G',
 '##AA',
 'befinden',
 'sich',
 'in',
 'Vor',
 '##räumen',
 'der',
 'Geschäfts',
 '##stellen',
 'der',
 'Banken',
 ',',
 'sodass',
 'sie',
 'auch',
 'außerhalb',
 'der',
 'Schalter',
 '##öffnungs',
 '##zeiten',
 'zugänglich',
 'sind',
 '.',
 '[SEP]']

In [6]:
tokenizer.convert_tokens_to_ids(tokens_num[0])

[102,
 1784,
 13183,
 28847,
 4686,
 618,
 261,
 4770,
 20815,
 136,
 261,
 21093,
 11600,
 387,
 397,
 2451,
 706,
 432,
 290,
 818,
 1398,
 1358,
 159,
 10695,
 3857,
 251,
 153,
 445,
 7721,
 125,
 2484,
 984,
 125,
 8232,
 818,
 7415,
 307,
 313,
 5729,
 125,
 28802,
 17893,
 4083,
 10370,
 341,
 566,
 103]

In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))
  scaler_input_word_ids = scaler.fit_transform(input_type_ids)  

  inputs = {
      #'sc': scaler_input_word_ids,
      #'input_word_ids': input_word_ids,
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [9]:
from sklearn.model_selection import train_test_split

x = loaded_set['Sentence']
y = loaded_set['MOS']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=32)
y_train = round(y_train, 2)
y_test = round(y_test, 2)
y_train


49     2.30
863    2.11
607    4.07
853    2.62
74     3.29
       ... 
88     1.89
892    1.31
310    3.20
555    2.11
727    4.00
Name: MOS, Length: 720, dtype: float64

In [6]:
import tensorflow as tf
X_train = bert_encode(x_train, tokenizer, max_seq_length)
X_test = bert_encode(x_test, tokenizer, max_seq_length)
X_train['input_word_ids']


<tf.Tensor: shape=(720, 144), dtype=int32, numpy=
array([[  102, 24507,  3518, ...,     0,     0,     0],
       [  102,   465,   125, ...,     0,     0,     0],
       [  102,   596,  3180, ...,     0,     0,     0],
       ...,
       [  102,   396,   125, ...,     0,     0,     0],
       [  102,  1130,  3484, ...,     0,     0,     0],
       [  102,   259,  2409, ...,     0,     0,     0]])>

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train['input_word_ids'] = scaler.fit_transform(X_train['input_word_ids'])
X_test['input_word_ids'] = scaler.transform(X_test['input_word_ids'])
n_timesteps = len(X_train['input_word_ids'])
n_features = len(X_train['input_word_ids'][0])
trainX = tf.reshape(X_train['input_word_ids'], [n_timesteps, n_features, 1]).numpy()
trainy = tf.reshape(y_train, [n_timesteps, 1]).numpy()

testX = tf.reshape(X_test['input_word_ids'], [len(X_test['input_word_ids']), n_features, 1]).numpy()
testy = tf.reshape(y_test, [len(X_test['input_word_ids']), 1]).numpy()
print(trainy.shape, trainX.shape, testX.shape, testy.shape)
trainX

(720, 1) (720, 144, 1) (180, 144, 1) (180, 1)


array([[[ 0.        ],
        [ 3.45053811],
        [-0.34845038],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [-0.52441672],
        [-0.76153954],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [-0.50275799],
        [-0.38960103],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       ...,

       [[ 0.        ],
        [-0.53582475],
        [-0.76153954],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [-0.41446975],
        [-0.3525898 ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [-0.55847548],
        [-0.48346832],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]]])

In [12]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model_conv = tf.keras.Sequential()
model_conv.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
model_conv.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
model_conv.add(tf.keras.layers.Dense(258, activation='relu'))
model_conv.add(tf.keras.layers.Dropout(0.1))
model_conv.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
model_conv.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))

model_conv.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 142, 64)           256       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 140, 32)           6176      
_________________________________________________________________
dense_3 (Dense)              (None, 140, 258)          8514      
_________________________________________________________________
dropout (Dropout)            (None, 140, 258)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 140, 1)            259       
Total params: 15,205
Trainable params: 15,205
Non-trainable params: 0
_________________________________________________________________


In [17]:
epochs =  20
batch_size = 12 
verbose = 1
model_conv.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)


Epoch 1/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 2/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 3/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 4/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 5/20
60/60 [==============================] - 0s 7ms/step - loss: 2.0544
Epoch 6/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 7/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 8/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 9/20
60/60 [==============================] - 1s 9ms/step - loss: 2.0544
Epoch 10/20
60/60 [==============================] - 0s 8ms/step - loss: 2.0544
Epoch 11/20
60/60 [==============================] - 0s 6ms/step - loss: 2.0544
Epoch 12/20
60/60 [==============================] - 0s 7ms/step - loss: 2.0544
Epoch 13/20
60/60 [==============================

In [170]:


epochs = 1
batch_size = 8  # select based on your GPU resources


train_data_size = len(y_train)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)




In [171]:
model.compile(optimizer=optimizer,
            loss='mse'
            )

In [172]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 144)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 144)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 144)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 768), (None, 177853441   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [173]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)

90/90 [==============================] - 521s 6s/step - loss: 5.5403 - val_loss: 5.0111


In [162]:
pred = model_conv.predict(testX)
rounded_pred = np.around(pred, decimals=2)
rounded_pred

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [187]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse(rounded_pred, y_test.values)

2.238544045678897